In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys

#HEADERS ={"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

In [ ]:
https://www.pycodemates.com/2022/01/Indeed-jobs-scraping-with-python-bs4-selenium-and-pandas.html

In [2]:
def get_test(url, job_title, location):
    driver = webdriver.Chrome("/Users/dswiss/Desktop/chromedriver")
    driver.get(url)
    time.sleep(3)
    driver.find_element_by_xpath('//*[@id="text-input-what"]').send_keys(job_title)
    time.sleep(3)
    locations = driver.find_element_by_xpath('//*[@id="text-input-where"]')
    if locations.get_attribute("value") == "":
        driver.find_element_by_xpath('//*[@id="text-input-where"]').send_keys(location)
    time.sleep(3)
    driver.find_element_by_xpath('/html/body/div').click()
    time.sleep(3)
    try:
        driver.find_element_by_xpath('//*[@id="jobsearch"]/button').click()
    except:
        driver.find_element_by_xpath('//*[@id="whatWhereFormId"]/div[3]/button').click()
    current_url = driver.current_url
    return current_url
    
current_url = get_test('https://indeed.com/','Data Scientist',"Los Angeles")

In [3]:
def scrape_job_details(url):
    
    resp = requests.get(url)
    content = BeautifulSoup(resp.content, 'lxml')
   
    jobs_list = []    
    for post in content.select('.job_seen_beacon'):
        try:
            data = {
                "job_title":post.select('.jobTitle')[0].get_text().strip(),
                "company":post.select('.companyName')[0].get_text().strip(),
                "rating":post.select('.ratingNumber')[0].get_text().strip(),
                "location":post.select('.companyLocation')[0].get_text().strip(),
                "date":post.select('.date')[0].get_text().strip(),
                "job_desc":post.select('.job-snippet')[0].get_text().strip(),
                "job_link":post.find(class_='jcs-JobTitle', href=True)['href']
            }
        except IndexError:
            continue          
        jobs_list.append(data)
    dataframe = pd.DataFrame(jobs_list)
    dataframe["job_link"] = 'https://www.indeed.com' + dataframe['job_link'].astype(str)
     
    return dataframe

lol = scrape_job_details(current_url)
lol["date"] = [x.replace('Posted','') for x in lol["date"]]
lol["job_title"] = [x.replace('new', '') for x in lol["job_title"]]

In [4]:
lol

,job_title,company,rating,location,date,job_desc,job_link
0,"Applied Scientist, AMP Data Science and Analyt...",Apple,4.2,"Culver City, CA+10 locations",3 days ago,Dive deep into large-scale data sources to unc...,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...
1,Data Scientist,Public Storage,3.3,"Glendale, CA",6 days ago,Strong command of applied math (linear/matrix ...,https://www.indeed.com/rc/clk?jk=5fda368a567f6...
2,Data Scientist - TikTok US - Tech Services,TikTok,3.2,"Los Angeles, CA+8 locations",30+ days ago,Understand underlying data sources and their l...,https://www.indeed.com/rc/clk?jk=8dce625b03da4...
3,Data Scientist - Predictive,Electronic Arts,3.8,"Los Angeles, CA",19 days ago,You will get hands on experience building tool...,https://www.indeed.com/rc/clk?jk=19ac3510766e3...
4,DATA SCIENTIST,Los Angeles County Department of Human Resources,4.1,"Los Angeles County, CA+3 locations",11 days ago,Four (4) years of experience applying machine ...,https://www.indeed.com/rc/clk?jk=6dd8d2a7c3fe7...
5,Data Scientist - Analytics,Twitch,4.4,"Remote in Los Angeles, CA",30+ days ago,Develop scalable data pipelines to support ana...,https://www.indeed.com/rc/clk?jk=6c15755a1bb6e...
6,Sr Data Scientist,Public Storage,3.3,"Glendale, CA",6 days ago,As part of the role is to lead and mentor data...,https://www.indeed.com/rc/clk?jk=f12500164aa96...
7,Data Scientist,Inspire,3.4,"Remote in Los Angeles, CA",Just posted,2-4 years of experience applying machine learn...,https://www.indeed.com/rc/clk?jk=ef91491c2402f...
8,Research Data Scientist - Meyer Lab - Computat...,CEDARS-SINAI,4.1,"Los Angeles, CA+4 locations",11 days ago,"Responsibilities include querying databases, d...",https://www.indeed.com/rc/clk?jk=89d9995769d01...
9,Data Scientist,REVOLVE,3.6,"Cerritos, CA 90703",Today,Knowledge of quantitative methods in statistic...,https://www.indeed.com/rc/clk?jk=b41926b0e0eb2...


In [6]:
lol.iloc[6]["job_link"]

'https://www.indeed.com/rc/clk?jk=f12500164aa96edc&fccid=212b3e869c8a74cb&vjs=3'